In [1]:
!wget https://raw.githubusercontent.com/Lavabar/kaggle_Herbarium22/main/efficientnet.py
!wget https://github.com/Lavabar/kaggle_Herbarium22/raw/2a66a174217fae8ac69da232f6e6dd5407faeeb0/checkpoint0_7000.pth

In [2]:
# PyTorch model and training necessities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from efficientnet import EfficientNetB0, EfficientNetB3

from torch.utils.tensorboard import SummaryWriter

# Image datasets and image manipulation
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

# Image display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image

torch.manual_seed(0)
images_path = '../input/herbarium-2022-fgvc9/train_images/'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [4]:
class FirstModelFeatureExtractor(object):
    
    def __init__(self, model_path):
        self.model = torch.load(model_path)
        self.model.eval()
        
    def __call__(self, sample):
        res = self.model.feature_extractor(torch.unsqueeze(sample, 0).cuda())
        #print(res.shape)
        return torch.squeeze(res)
        

In [5]:
# Gather datasets and prepare them for consumption
transform = transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                                FirstModelFeatureExtractor('./checkpoint0_7000.pth')
                            ])

class CategoryDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe_X, dataframe_Y, transform, keys):
        self.dataframe = dataframe_X.merge(dataframe_Y)
        self.transform = transform
        self.keys = keys

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        return (
            transform(Image.open(images_path + row["file_name"])).cuda(),
            torch.tensor(self.keys[row["category_id"]]).cuda()
        )

In [6]:
def do_training(training_loader, validation_loader, net, family_id, n_epoch=1):
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(net.parameters(), lr=0.001)
    
    for epoch in range(n_epoch):  # loop over the dataset multiple times
        running_loss = 0.0

        for i, data in enumerate(training_loader, 0):
            # basic training loop
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print('{}, Batch {}'.format(family_id, i + 1))
            running_loss += loss.item()
            if i % 50 == 49:# Every 50 mini-batches...
                torch.save(net, f'./checkpoint{family_id}_{epoch}_{i+1}.pth')
                # Check against the validation set
                running_vloss = 0.0

                net.train(False) # Don't need to track gradents for validation
                for j, vdata in enumerate(validation_loader, 0):
                    vinputs, vlabels = vdata
                    voutputs = net(vinputs)
                    vloss = criterion(voutputs, vlabels)
                    running_vloss += vloss.item()
                net.train(True) # Turn gradients back on for training

                avg_loss = running_loss / 50
                avg_vloss = running_vloss / len(validation_loader)

                # Log the running loss averaged per batch
                writer.add_scalars('Training vs. Validation Loss',
                                { 'Training' : avg_loss, 'Validation' : avg_vloss },
                                epoch * len(training_loader) + i)

                running_loss = 0.0
    print('Finished Training')
    writer.flush()
    
    return net

In [7]:
import os
fam_ids = sorted(os.listdir('../input/herbarium22-family-split/dataset_fsplit/'))

In [ ]:
for family_id in fam_ids[101:]:

    train_X = pd.read_csv(f'../input/herbarium22-family-split/dataset_fsplit/{family_id}/train_X.csv')
    validation_X = pd.read_csv(f'../input/herbarium22-family-split/dataset_fsplit/{family_id}/valid_X.csv')
    train_Y = pd.read_csv(f'../input/herbarium22-family-split/dataset_fsplit/{family_id}/train_Y.csv')
    validation_Y = pd.read_csv(f'../input/herbarium22-family-split/dataset_fsplit/{family_id}/valid_Y.csv')

    n_categories = train_Y.nunique()['category_id']

    keys = dict(zip(sorted(train_Y['category_id'].unique()), range(n_categories)))

    train_dataset = CategoryDataset(train_X, train_Y, transform, keys)
    validation_dataset = CategoryDataset(validation_X, validation_Y, transform, keys)

    training_loader = torch.utils.data.DataLoader(train_dataset,
                                                  batch_size=64,
                                                  shuffle=True)


    validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                    batch_size=8,
                                                    shuffle=False)

    net = EfficientNetB0(in_sz=1536, out_sz=n_categories)
    net.to(device)


    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter(f'runs/herb_experiment_f{family_id}')

    net = do_training(training_loader, validation_loader, net, family_id)
    torch.save(net, f'./checkpoint_{family_id}_final.pth')

In [ ]:
!zip -r -9 models_.zip ./checkpoint_*_final.pth